# Prophet demo

## Preamble

In [23]:
import pandas
import seaborn
import matplotlib
import forecast_lab
import sklearn

In [24]:
seaborn.set_style("ticks")
matplotlib.pyplot.rcParams["axes.grid"] = True
matplotlib.pyplot.rcParams["figure.figsize"] = (20, 3)

In [26]:
taxi_trips = forecast_lab.datasets.read_chicago_taxi_trips_daily()

In [27]:
taxi_trips.head()

,Trips
Date,
2013-01-01,96
2013-01-02,53
2013-01-03,61
2013-01-04,79
2013-01-05,76


Prophet expects the input time series in the format of a `pandas.DataFrame` with columns `ds` for time and `y` for values.

In [39]:
taxi_trips_p  = pandas.DataFrame(taxi_trips).reset_index()
taxi_trips_p.columns = ["ds", "y"]
superbowls = pandas.DataFrame({
  'holiday': 'superbowl',
  'ds': pandas.to_datetime([
'2013-02-03',
'2014-02-02',
'2015-02-01',
'2016-02-07',
'2017-02-05'
  ]),
  'lower_window': 0,
  'upper_window': 1,
})
## Evaluation

In [65]:
forecast_lab.ForecastEvaluation(
        ts=taxi_trips["Trips"],
        forecasting=forecast_lab.ProphetWrapper(
            freq="d",
            estimator_params={
                "holidays_prior_scale": 20,
                "holidays": superbowls,
                "mcmc_samples": 0,
                "yearly_seasonality": 10,
                "weekly_seasonality": True}),
        test_window_size=30,
        train_window_size=365,
        metrics={
            "MAE": sklearn.metrics.mean_absolute_error
        }
).evaluate(
    k=20,
    plot_segments=True,
    plot_residuals=True,
).get_metrics().mean()

AttributeError: module 'forecast_lab.metrics' has no attribute 'mean_absolute_error_local'

In [ ]:
forecast_lab.ForecastEvaluation(
        ts=taxi_trips["Trips"],
        forecasting=forecast_lab.dummy.MeanForecast(),
        test_window_size=30,
        train_window_size=365,
        metrics={
            "MAE": sklearn.metrics.mean_absolute_error
        }
).evaluate(
    k=20,
    plot_segments=True,
    plot_residuals=True,
).get_metrics().mean()

## References
